# Algoritmos de busqueda

La busqueda es una operación **escencial** que consiste en encontrar un elemento dentro de una colección de datos.

## Busqueda lineal

In [2]:
def linear_search(list, item):
    """Busca un ítem en una lista usando el algoritmo de búsqueda lineal. Este algoritmo revisa cada elemento en secuencia (de principio a fin)
    hasta que encuentra una coincidencia.

    Args:
        list (list): La lista (o secuencia) en la que se va a buscar.
        item (any): El valor que se desea encontrar.

    Returns:
        int: El índice de la *primera* ocurrencia del ítem en la lista.
             Retorna -1 si el ítem no se encuentra.
    """
    for i in range(len(list)):
        if list[i] == item:
            return i
    return -1

A continuación vamos a realizar una prueba sencilla.

In [5]:
L = [3, 5, 2, 4, 9, 6, 1, 8, 7]
print(linear_search(L, 4))  # Salida: 3
print(linear_search(L, 10)) # Salida: -1

3
-1


## Busqueda binaria

In [6]:
def binary_search(sorted_list, item):
    """
    Busca un ítem en una lista "ordenada" usando el algoritmo de búsqueda binaria.

    Args:
        sorted_list (list): La lista (o secuencia) en la que se va a buscar.
                            Debe estar previamente ordenada.
        item (any): El valor que se desea encontrar.

    Returns:
        int: El índice del ítem en la lista.
             Retorna -1 si el ítem no se encuentra.
    """
    low = 0
    high = len(sorted_list) - 1
    while low <= high:
        mid = (low + high) // 2
        guess = sorted_list[mid]
        if guess == item:
            return mid
        if guess > item:
            high = mid - 1
        else:
            low = mid + 1
    return -1

A continuación vamos a realizar una prueba sencilla.

In [7]:
L = [1, 2, 3, 4, 5, 6, 7, 8, 9]
print(binary_search(L, 4))  # Salida: 3
print(binary_search(L, 10)) # Salida: -1

3
-1


## Medicion del desempeño

### 1. Peparar los datos
Se emplea un $N$ grande y un escenario de "peor caso".
* $N = 1000000$
* $item = -1$ (Valor que no esta)

In [8]:
# N = 1 millón de elementos
N = 1_000_000 

# 'data' es una lista ordenada de 0 a 999,999
data = list(range(N))

# El objetivo es -1 (sabemos que no está en la lista)
target = -1

### Ejecutar el Test de Rendimiento

Use `%timeit` en celdas separadas para medir cada algoritmo.

In [10]:
# Mide el tiempo de linear_search
%timeit linear_search(data, target)

97.9 ms ± 15 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
# Mide el tiempo de binary_search
%timeit binary_search(data, target)

5.63 μs ± 737 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


### Analisis de resultados

Para N = 1000000:
* **Lineal**: 97.9ms
* **Binaria**: 5.63us
* **Comparacion**:
  
  $$
  \frac{T_L}{T_B} = \frac{97.9m}{5.63} = 1702381.88
  $$


**La Prueba de Duplicación**: Repitamos el procedimiento duplicando la cantidad de datos.

In [13]:
# N = 2 millón de elementos
N = 2_000_000 

# 'data' es una lista ordenada de 0 a 1,999,999
data = list(range(N))

# El objetivo es -1 (sabemos que no está en la lista)
target = -1

In [18]:
# Mide el tiempo de linear_search
%timeit linear_search(data, target)

155 ms ± 28.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
# Mide el tiempo de binary_search
%timeit binary_search(data, target)

4.65 μs ± 482 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


Para N = 2000000 tenemos los siguientes tiempos:
* **Lineal**: 155ms
* **Binaria**: 4.55us

## Prueba de duplicación para varios valores

In [ ]:
import time
import math

### Clase Stopwatch
class Stopwatch:
    """Mide el tiempo transcurrido usando el reloj monotónico de alta precisión."""
    def __init__(self):
        # Usar perf_counter() para rendimiento
        self._startTime = time.perf_counter()

    def elapsedTime(self):
        """Retorna el tiempo transcurrido en segundos."""
        return time.perf_counter() - self._startTime

In [22]:
# --- Script Principal del Test ---

def run_doubling_test(algorithm_to_test, start_N=1000, max_N=100_000_000):
    """
    Ejecuta el test de duplicación para un algoritmo de búsqueda dado.
    """
    print(f"--- Iniciando Prueba de Duplicación para: {algorithm_to_test.__name__} ---")
    print(f"{'N':>12s} | {'Tiempo (s)':>15s} | {'Ratio':>10s}")
    print("-" * 41)
    
    N = start_N
    tiempo_previo = 0.0

    while N <= max_N:
        
        # 1. Preparar los datos (lista ordenada de 0 a N-1)
        # Se genera CADA vez para medir la creación si es relevante
        # o para evitar problemas de caché.
        datos = list(range(N))
        
        # 2. Definir el PEOR CASO (ítem que no existe)
        objetivo_peor_caso = -1 
        
        # 3. Medir el tiempo de ejecución
        watch = Stopwatch()
        
        # ¡Ejecutamos el algoritmo!
        algorithm_to_test(datos, objetivo_peor_caso)
        
        tiempo_actual = watch.elapsedTime()
        
        # 4. Calcular el Ratio (si es posible)
        ratio_str = "N/A"
        # Solo calculamos el ratio si el tiempo previo es medible (no es ruido)
        if tiempo_previo > 0.000001: 
            ratio = tiempo_actual / tiempo_previo
            ratio_str = f"{ratio:10.2f}"
        
        # 5. Imprimir la tabla
        print(f"{N:12d} | {tiempo_actual:15.6f} | {ratio_str}")
        
        # 6. Duplicar N para la siguiente iteración
        N = N * 2
        tiempo_previo = tiempo_actual
        
        # Si el tiempo es demasiado largo, parar (seguro para O(N^2) o O(N^3))
        if tiempo_actual > 10.0: # 10 segundos
            print("El tiempo de ejecución es demasiado largo, deteniendo prueba.")
            break

In [23]:
# Prueba 1: Linear Search (O(N))
run_doubling_test(linear_search, start_N=100_000, max_N=100_000_000)

--- Iniciando Prueba de Duplicación para: linear_search ---
           N |      Tiempo (s) |      Ratio
-----------------------------------------
      100000 |        0.018146 | N/A
      200000 |        0.039432 |       2.17
      400000 |        0.049308 |       1.25
      800000 |        0.086888 |       1.76
     1600000 |        0.185581 |       2.14
     3200000 |        0.272488 |       1.47
     6400000 |        0.619268 |       2.27
    12800000 |        1.163509 |       1.88
    25600000 |        2.787359 |       2.40
    51200000 |        5.007819 |       1.80


In [26]:
# Prueba 2: Binary Search (O(log N))
# (¡Necesitamos Ns mucho más grandes para esta!)
run_doubling_test(binary_search, start_N=100_000, max_N=100_000_000)

--- Iniciando Prueba de Duplicación para: binary_search ---
           N |      Tiempo (s) |      Ratio
-----------------------------------------
      100000 |        0.000021 | N/A
      200000 |        0.000023 |       1.13
      400000 |        0.000026 |       1.09
      800000 |        0.000023 |       0.90
     1600000 |        0.000026 |       1.12
     3200000 |        0.000027 |       1.03
     6400000 |        0.000023 |       0.86
    12800000 |        0.000020 |       0.88
    25600000 |        0.000029 |       1.45
    51200000 |        0.000022 |       0.76


## Refererencias

1. https://algs4.cs.princeton.edu/14analysis/
2. https://introcs.cs.princeton.edu/python/23recursion/
3. https://www.cs.princeton.edu/courses/archive/spring21/cos226/lectures/study/14AnalysisOfAlgorithms.html